In [ ]:
from data import generate_mixture_gaussians, generate_happy_face
from plotting_functions import plot_points
import jax

%load_ext autoreload
%autoreload 2

jax.config.update("jax_enable_x64", True)

x=generate_happy_face(1000)
y=generate_mixture_gaussians(1000)

plot_points(x)
plot_points(y)

In [ ]:
from sde import BridgeVPSDE
import jax.numpy as jnp
import jax.random as jr
from plotting_functions import plot_points_over_time


sde=BridgeVPSDE(.1, 1)

def generate_steps(x, y, times, key=jr.PRNGKey(0) ,T=1):
   
    # Initialize an array to store the generated points
    data = jnp.zeros((x.shape[0], 2, len(times)))
    
    # Loop over each time step
    for i, t in enumerate(times):

        key, subkey = jr.split(key)

        mean, std = sde.p(x, y, t, T)
        
        # Generate points using the normal distribution
        noise = jr.normal( subkey, x.shape)
        sample=mean + std * noise
        
        data = data.at[:, :, i].set(sample)  
        
    return data

# Example usage
times = jnp.linspace(0, 1, 12)  # 10 time steps from 0 to 1
data = generate_steps(x, y, times)
plot_points_over_time(data, times)


In [ ]:
from Score_nets import Bridge_Diffusion_Net, UNet
import jax.random as jr

key=jr.PRNGKey(0)

UNet_Example=UNet(
        key=key,
        data_shape=(2,28,28),
        is_biggan=False,
        dim_mults=[1, 2, 4],
        hidden_size=64,
        heads=4,
        dim_head=32,
        dropout_rate=0.0,
        num_res_blocks=2,
        attn_resolutions=[16],
 
    )

model=Bridge_Diffusion_Net(key, 4,2)
print(model)

In [ ]:

from train import main
import equinox as eqx

%load_ext autoreload
%autoreload 2

key = jr.PRNGKey(123)
x=generate_mixture_gaussians(100_000)
y=generate_happy_face(100_000)


model = eqx.tree_deserialise_leaves("checkpoints/Bridge/2d_bmin=.01_bmax=1.eqx", model)
main(model,
      x,
      sde,
      data_y=y,
      lr=1e-4,
      print_every=1000,
      filename="2d_bmin=.01_bmax=1"
      )